In [1]:
# pip install flask

In [2]:
# importing required libraries
import pandas as pd
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [3]:
# read the dataset
data = pd.read_csv('twitter_sentiments.csv')
# view the top rows
data.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [4]:
# train test split
train, test = train_test_split(data, test_size = 0.2, stratify = data['label'], random_state=21)

# get the shape of train and test split.
train.shape, test.shape
## >> ((25569, 3), (6393, 3))

((25569, 3), (6393, 3))

In [5]:
# create a TF-IDF vectorizer object
tfidf_vectorizer = TfidfVectorizer(lowercase= True, max_features=1000, stop_words=ENGLISH_STOP_WORDS)

# fit the object with the training data tweets
tfidf_vectorizer.fit(train.tweet)

TfidfVectorizer(max_features=1000,
                stop_words=frozenset({'a', 'about', 'above', 'across', 'after',
                                      'afterwards', 'again', 'against', 'all',
                                      'almost', 'alone', 'along', 'already',
                                      'also', 'although', 'always', 'am',
                                      'among', 'amongst', 'amoungst', 'amount',
                                      'an', 'and', 'another', 'any', 'anyhow',
                                      'anyone', 'anything', 'anyway',
                                      'anywhere', ...}))

In [6]:
# transform the train and test data
train_idf = tfidf_vectorizer.transform(train.tweet)
test_idf  = tfidf_vectorizer.transform(test.tweet)

In [7]:
# create the object of LinearRegression Model
model_LR = LogisticRegression()

# fit the model with the training data
model_LR.fit(train_idf, train.label)

# predict the label on the traning data
predict_train = model_LR.predict(train_idf)

# predict the model on the test data
predict_test = model_LR.predict(test_idf)

# f1 score on train data
f1_score(y_true= train.label, y_pred= predict_train)
## >> 0.4888178913738019

f1_score(y_true= test.label, y_pred= predict_test)
## >> 0.45751633986928114

0.45751633986928114

In [8]:
# define the stages of the pipeline
pipeline = Pipeline(steps= [('tfidf', TfidfVectorizer(lowercase=True,
                                                      max_features=1000,
                                                      stop_words= ENGLISH_STOP_WORDS)),
                            ('model', LogisticRegression())])

# fit the pipeline model with the training data                            
pipeline.fit(train.tweet, train.label)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=1000,
                                 stop_words=frozenset({'a', 'about', 'above',
                                                       'across', 'after',
                                                       'afterwards', 'again',
                                                       'against', 'all',
                                                       'almost', 'alone',
                                                       'along', 'already',
                                                       'also', 'although',
                                                       'always', 'am', 'among',
                                                       'amongst', 'amoungst',
                                                       'amount', 'an', 'and',
                                                       'another', 'any',
                                                       'anyhow', 'anyone',
                                                       'anything', 'anyway',
                                                       'anywhere', ...}))),
                ('model', LogisticRegression())])

In [9]:
# sample tweet
text = ["Virat Kohli, AB de Villiers set to auction their 'Green Day' kits from 2016 IPL match to raise funds"]

# predict the label using the pipeline
pipeline.predict(text)
## >> array([0])

array([0], dtype=int64)

In [10]:
# import joblib
from joblib import dump

In [11]:
# dump the pipeline model
dump(pipeline, filename="text_classification.joblib")

['text_classification.joblib']

In [12]:
# import joblib
from joblib import load

# sample tweet text
text = ["Virat Kohli, AB de Villiers set to auction their 'Green Day' kits from 2016 IPL match to raise funds"]

# load the saved pipleine model
pipeline = load("text_classification.joblib")

# predict on the sample tweet text
pipeline.predict(text)
## >> array([0])

array([0], dtype=int64)

In [13]:
# !pip3 install tweepy

In [14]:
# import required libraries
import tweepy
import time
import pandas as pd
pd.set_option('display.max_colwidth', 1000)

# api key
api_key = "xxxx"
# api secret key
api_secret_key = "xxxx"
# access token
access_token = "xxxx"
# access token secret
access_token_secret = "xxxx"

# authorize the API Key
authentication = tweepy.OAuthHandler(api_key, api_secret_key)

# authorization to user's access token and access token secret
authentication.set_access_token(access_token, access_token_secret)

# call the api
api = tweepy.API(authentication, wait_on_rate_limit=True)

In [15]:
def get_related_tweets(text_query):
    # list to store tweets
    tweets_list = []
    # no of tweets
    count = 50
    try:
        # Pulling individual tweets from query
        for tweet in api.search_tweets(q=text_query, count=count):
            print(tweet.text)
            # Adding to list that contains all tweets
            tweets_list.append({'created_at': tweet.created_at,
                                'tweet_id': tweet.id,
                                'tweet_text': tweet.text})
        return pd.DataFrame.from_dict(tweets_list)

    except BaseException as e:
        print('failed on_status,', str(e))
        time.sleep(3)

In [16]:
from get_tweets import get_related_tweets

In [17]:
# importing the required libraries
from flask import Flask, render_template, request, redirect, url_for
from joblib import load


# load the pipeline object
pipeline = load("text_classification.joblib")

# function to get results for a particular text query
def requestResults(name):
    # get the tweets text
    tweets = get_related_tweets(name)
    # get the prediction
    tweets['prediction'] = pipeline.predict(tweets['tweet_text'])
    # get the value counts of different labels predicted
    data = str(tweets.prediction.value_counts()) + '\n\n'
    return data + str(tweets)

In [18]:
# start flask
app = Flask(__name__)

# render default webpage
@app.route('/')
def home():
    return render_template('home.html')

# when the post method detect, then redirect to success function
@app.route('/', methods=['POST', 'GET'])
def get_data():
    if request.method == 'POST':
        user = request.form['search']
        return redirect(url_for('success', name=user))

# get the data for the requested query
@app.route('/success/<name>')
def success(name):
    return "<xmp>" + str(requestResults(name)) + " </xmp> "

In [ ]:
app.run(debug=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Dec/2022 22:34:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2022 22:34:40] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [04/Dec/2022 22:34:42] "GET /success/Elon%20Musk HTTP/1.1" 200 -


RT @jamisonfoser: It has been 22 hours since Matt Taibbi posted the personal email address of a member of congress in a thread promoted by…
RT @vanessavallej0: “¡Si cometo suicidio no es real!”

Elon Musk acaba de decir esto en un Space en Twitter. Parece que el nuevo dueño de l…
@IanJaegerNPC Elon Musk’s “big scoop”: Basically the Biden campaign team (not the Government) asked Twitter (a priv… https://t.co/7zjdII4Hc9
RT @lordivan22: Brasil 🇧🇷: A mídia internacional repercute a declaração feita por Elon  Musk, de que funcionários do Twitter no Brasil, pod…
RT @lordivan22: Brasil 🇧🇷: A mídia internacional repercute a declaração feita por Elon  Musk, de que funcionários do Twitter no Brasil, pod…
RT @CNN: Two watchdog groups have published research that indicates Elon Musk’s claim that hate speech dramatically decreased on Twitter si…
RT @rmotta2: Conforme prometido, segue a primeira parte dos “Arquivos do Twitter”, cuja publicação foi determinada por Elon Musk, e que voc…
Elon Musk's Co

127.0.0.1 - - [04/Dec/2022 22:34:42] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/Dec/2022 22:36:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2022 22:41:22] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [04/Dec/2022 22:41:23] "GET /success/elon%20musk HTTP/1.1" 200 -


RT @lordivan22: Brasil 🇧🇷: A mídia internacional repercute a declaração feita por Elon  Musk, de que funcionários do Twitter no Brasil, pod…
RT @OzraeliAvi: #BREAKING: Elon Musk confirms he is NOT suicidal in case they Epstein him.
RT @JayneZirkle: Brazilian Leftist Censorship Exposed: Jayne Zirkle Explains Concerning Revelations From Elon Musk On The United States' Br…
RT @mirandadevine: Column’s up: What’s missing from Elon Musk’s #TwitterFiles: the truth about the FBI https://t.co/AtkUCgOatN
Elon Musk blasted the New York Times Saturday for its refusal to cover the ongoing firestorm over political censors… https://t.co/mB8XmRadiR
Elon Musk geldi, onlar gitti: Twitter'ı terk eden ünlüler https://t.co/BwoSBVWESy https://t.co/h4KpGxapb7
RT @Reuters: ICYMI: Twitter CEO Elon Musk said the social media platform saw 'all-time high' new user signups in the seven days to November…
RT @davidmweissman: I know I have asked Elon Musk a couple of times to do his job enforcing Twitter standards by